In [1]:
import matplotlib.pyplot as plt
import tree_utils as tu
from tqdm import tqdm
import networkx as nx
import numpy as np
import random
from sklearn.preprocessing import normalize

"""
Note:
The frontier is strictly suppposed to be used as edges only. There will be straggling nodes lying around that 
you need to be careful for. 

This is especially important for concatenating graphs. in stead of combining, it is best to place the edges into
a new graph one by one.
"""


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


'\nNote:\nThe frontier is strictly suppposed to be used as edges only. There will be straggling nodes lying around that \nyou need to be careful for. \n\nThis is especially important for concatenating graphs. in stead of combining, it is best to place the edges into\na new graph one by one.\n'

In [2]:
dims = (2, 2)
g = tu.generate_grid_graph(dims, queen=True)

# relabel nodes if you want them as {1, 2, 3,...} instead of {(0,0), (0, 1), ...}
# def reverse_dict(d):
#     new_dict = dict()
#     for k, v in d.items():
#         new_dict[v] = k
#     return new_dict
# labels = {node: (dims[0] * node[0]) + node[1] + 1 for node in g.nodes}
# g = nx.relabel_nodes(g, labels, copy=False)
# node_map = reverse_dict(labels)
# tu.draw(g, node_map=node_map)

g = nx.relabel_nodes(g, {node: (dims[0] * node[0]) + node[1] + 1 for node in g.nodes}, copy=False)
# tu.draw(g)

# queen_trees = sample_trees(queen, tu.uniform_random_spanning_tree, 100000)
trees = tu.enumerate_all_trees(g)

ds = [0]
tree_dict = tu.make_tree_dict(trees, ds)
counting_dict = tu.make_counting_dict(g, tree_dict, ds)

16it [00:00, 5342.21it/s]
100%|██████████████████████████████████████████████████████████████████████| 16/16 [00:00<00:00, 46538.74it/s]


In [3]:
def valid_run(snapped_edge, cc1, cc2):
    u, v = snapped_edge
    g = nx.Graph()
    g.add_edge(u, v)
    
    for edge in cc1.edges:
        u, v = edge
        g.add_edge(u, v)
        
    for edge in cc2.edges:
        u, v = edge
        g.add_edge(u, v)
        
    return nx.is_tree(g), g     

def get_ccs(g, counting_dict, snapped_edge):
    """
    """
    # u, v = tu.sample_edge_from_counts(counting_dict)
    cc1, cc2, cc1_frontier, cc2_frontier = tu.initialize(g, snapped_edge)
    target_size = len(g.nodes)/2

    while not tu.trees_correct_sizes(cc1, cc2, target_size):
        if tu.is_bottlenecked(cc1, cc1_frontier, target_size):
            tu.break_bottleneck(g, cc1, cc2, snapped_edge, cc1_frontier, cc2_frontier)

        if len(cc1.nodes) != target_size:
            tu.sample_from_frontier_to_cc(g, cc1_frontier, cc1, cc2_frontier, cc2)

        if tu.is_bottlenecked(cc2, cc2_frontier, target_size):
            tu.break_bottleneck(g, cc2, cc1, snapped_edge, cc2_frontier, cc1_frontier)

        if len(cc2.nodes) != target_size:
            tu.sample_from_frontier_to_cc(g, cc2_frontier, cc2, cc1_frontier, cc1)
    
    result, tree = valid_run(snapped_edge, cc1, cc2)
    return tree
    
edge_map = dict()
for i, edge in enumerate(g.edges):
    edge_map[edge] = i
    
tree_counter = dict()

num_runs = 10000
for i in range(num_runs):
    tree = get_ccs(g, counting_dict, (3, 4))
    edge_set = tuple(sorted([edge_map[tuple(sorted(edge))] for edge in tree.edges]))
    if edge_set in tree_counter.keys():
        tree_counter[edge_set] += 1
    else:
        tree_counter[edge_set] = 1


In [4]:
for k, v in tree_counter.items():
    tree_counter[k] = v/ num_runs

In [5]:
tree_counter

{(2, 3, 5): 0.5152, (1, 4, 5): 0.4848}

In [1]:
"""
Below are junk algorithms, mostly:
"""

'\nBelow are junk algorithms, mostly:\n'

In [3]:
"""
The properties of the boundary are that it only contains the edges that are in the xternal 
boundary of the branch, and contain no edges from the branch itself.
"""

def compute_boundary(g, branch):
    """ Computes the external boundary of the branch. This consists of boundary 
    """
    boundary = nx.Graph()
    for node in branch.nodes:
        for neighbor in g.neighbors(node):
            if neighbor not in branch.edges:
                boundary.add_edge(node, neighbor)
    return boundary
    
def add_connected_component(g, tree):
    """
    """
    leaf = pick_leaf(g, tree)
    branch = nx.Graph()
    branch.add_node(leaf)
    boundary = compute_boundary(g, branch)
    reached_tree = False
    
    while reached_tree is False:
        nxt = random.choice(list(boundary.edges))  
        
        branch.add_edge(nxt[0], nxt[1])
        
        if loop_exists(branch):
#             print("Loop exists!")
            branch.remove_edge(nxt[0], nxt[1])
            erase_singletons(branch)
        else:
            boundary = compute_boundary(g, branch)
#             print("Boundary edges after increase: ", boundary.edges)

        if nxt[0] in tree.nodes or nxt[1] in tree.nodes:
#             print("Reached Tree")
            reached_tree = True
            
    tree = tree.update(branch.edges)
    
def wilsons_modified(g):
    root = randomly_pick_root(g)
#     print("Root: ", root)
    tree = nx.Graph()
    tree.add_node(root)
    
    while tree.number_of_nodes() < g.number_of_nodes():
        add_connected_component(g, tree)
#         tu.draw(tree, delay=2)
#         print()
    return tree

# tree = wilsons_modified(g)
# tu.draw(tree)

In [4]:
def compute_boundary_node(g, branch):
    boundary = set()
    for node in branch.nodes:
        for neighbor in g.neighbors(node):
            if neighbor not in branch.nodes:
                boundary.add(node)
    return boundary

def add_connected_component_new(g, tree):
    """
    """
    leaf = pick_leaf(g, tree)
    branch = nx.Graph()
    branch.add_node(leaf)
    boundary = compute_boundary_node(g, branch)
    reached_tree = False
    
    while reached_tree is False:
        nxt_src = random.choice(list(boundary))
        nxt_dst = random.choice(list(g.neighbors(nxt_src)))
        
        branch.add_edge(nxt_src, nxt_dst)
        
        if loop_exists(branch):
#             print("Loop exists!")
            branch.remove_edge(nxt_src, nxt_dst)
            erase_singletons(branch)
        else:
            boundary = compute_boundary_node(g, branch)
#             print("Boundary edges after increase: ", boundary.edges)

        if nxt_src in tree.nodes or nxt_dst in tree.nodes:
#             print("Reached Tree")
            reached_tree = True
            
    tree = tree.update(branch.edges)
    
def wilsons_modified_node(g):
    root = randomly_pick_root(g)
#     print("Root: ", root)
    tree = nx.Graph()
    tree.add_node(root)
    
    while tree.number_of_nodes() < g.number_of_nodes():
        add_connected_component_new(g, tree)
#         tu.draw(tree, delay=2)
#         print()
    return tree

# num_trials = 100000
# data = []

# for _ in tqdm(range(num_trials)):
#     tree = wilsons_modified_node(g)

#     idx = all_trees.index(tu.tup(tree))
#     data.append(idx)
    
# tu.plot_sampled_STs(all_trees, data)

In [5]:
def erase_loopmaker_if_exists(tree, curr, nxt):
    try:
        cycle = nx.find_cycle(tree)
        tree.remove_edge(curr, nxt)
        erase_singletons(tree)
        return True
    except nx.exception.NetworkXNoCycle:
        return False
    
def cut_edges(g, tree):
    """ returns the set of cut edges of a graph
    """
    cuts = []
    for (u, v) in g.edges: 
        if u not in nx.node_connected_component(tree, v):
            cuts.append((u, v))
    return cuts
    
def add_connected_component_og(g, tree):
    """
    """
    leaf = pick_leaf(g, tree)
    curr = leaf
    
    hit_tree = False
    loopmaker = False
        
    while hit_tree is False and loopmaker is False:
        nxt = random.choice(list(g.neighbors(curr)))
            
        if tree.has_edge(curr, nxt):
            loopmaker = True
            continue
            
        tree.add_edge(curr, nxt)
        loopmaker = erase_loopmaker_if_exists(tree, curr, nxt)
        curr = nxt

        if nxt in tree.nodes():
            hit_tree = True
            

def og(g):
#     root = randomly_pick_root(g)
#     print("Root: ", root)
    tree = nx.Graph()
#     tree.add_node(root) 
    
    while tree.number_of_nodes() < g.number_of_nodes():
        add_connected_component_og(g, tree)
#         tu.draw(tree, delay=2)
#         print()
    while not nx.is_connected(tree):
#         print(len(tree.edges))
        cuts = cut_edges(g, tree)
#         print(cuts)
        edge = random.choice(cuts)
        tree.add_edge(edge[0], edge[1])
        
    return tree


# num_trials = 100000
# data = []

# for _ in tqdm(range(num_trials)):
#     tree = og(g)

#     idx = all_trees.index(tu.tup(tree))
#     data.append(idx)
    
# tu.plot_sampled_STs(all_trees, data)

In [6]:
"""
randomly pick a vertex in the loop and direct it to a new direction
"""
def nodes_from_cycle(cycle):
    nodes = set()
    for (u, v) in cycle:
        nodes.add(u)
        nodes.add(v)
    return list(nodes)

def add_branch_new(g, tree):
    leaf = pick_leaf(g, tree)
    curr = leaf
    branch = nx.Graph()
    reached_root = False
    
    while reached_root is False:
        nxt = random.choice(list(g.neighbors(curr)))
        
        branch.add_edge(curr, nxt)
        while loop_exists(branch):
            print("looping")
            print("Branch: ", branch.edges)
            cycle = nx.find_cycle(branch)
            print("Cycle: ", cycle)
            curr = random.choice(nodes_from_cycle(cycle))
            nxt = random.choice(list(g.neighbors(curr)))
            branch.add_edge(curr, nxt)
            print("curr, nxt: ", curr, nxt)
        curr = nxt
        
        if nxt in tree.nodes():
            reached_root = True
            
    tree = tree.update(branch.edges, branch.nodes)
    
def wilsons_new(g):
    root = randomly_pick_root(g)
    tree = nx.Graph()
    tree.add_node(root)
    
    while tree.number_of_nodes() < g.number_of_nodes():
        add_branch_new(g, tree)
        print("added branch")
    return tree

In [1]:
# reverse wilsons stuff...

In [2]:
def get_frontier(g, tree):
    frontier = set()
    for node in tree.nodes:
        for neighbor in g.neighbors(node):
            frontier.add(neighbor)
    frontier = frontier.difference(tree.nodes)
    return list(frontier)

def get_weighted_frontier(g, tree):
    frontier = set()
    for node in tree.nodes:
        for neighbor in g.neighbors(node):
            frontier.add(neighbor)
            
    frontier = frontier.difference(tree.nodes)
    
    weighted_frontier = []
    for node in frontier:
        for neighbor in g.neighbors(node):
            if neighbor in tree.nodes:
                weighted_frontier.append(node)
    return weighted_frontier

def get_inverse_weighted_frontier(g, tree):
    frontier = set()
    for node in tree.nodes:
        for neighbor in g.neighbors(node):
            frontier.add(neighbor)
            
    frontier = frontier.difference(tree.nodes)
    
    weighted_frontier = []
    for node in frontier:
        for neighbor in g.neighbors(node):
            if neighbor in tree.nodes:
                weighted_frontier.append(node)
    
    counts = {}
    for item in weighted_frontier:
        if item in counts.keys():
            counts[item] += 1
        else:
            counts[item] = 1
    
    weights = []
    nodes = []
    for key, value in counts.items():
        nodes.append(node)
        weights.append(value)
        
    sample = random.choices(nodes, weights=weights, k=1)[0]
    return sample
                 
def pick_neighbor_in_tree(g, tree, node):
    neighbors = []
    for neighbor in g.neighbors(node):
        if neighbor in tree.nodes:
            neighbors.append(neighbor)
    
    return random.choice(neighbors)
    
def reverse_wilsons(g):
    root = randomly_pick_root(g)
    
    tree = nx.Graph()
    tree.add_node(root)
    
    while tree.number_of_nodes() < g.number_of_nodes():
#         frontier = get_frontier(g, tree)
#         sample = random.choice(frontier)
        sample = get_inverse_weighted_frontier(g, tree)
        neighbor = random.choice(list(g.neighbors(sample)))

        if neighbor in tree.nodes:
            tree.add_edge(sample, neighbor)
        
    return tree

for i in range(1000):
    tree = reverse_wilsons(g)
    test_valid_tree(tree, g)
    
num_trials = 100000
data = []

for _ in tqdm(range(num_trials)):
    tree = reverse_wilsons(g)

    idx = all_trees.index(tu.tup(tree))
    data.append(idx)
    
tu.plot_sampled_STs(all_trees, data)

NameError: name 'g' is not defined

In [3]:
def initialize_stacks(g, root):
    stacks = dict()
    
    for node in g.nodes():
        if node == root:
            continue
        stacks[node] = random.choice(list(g.neighbors(node)))
        
    return stacks

def reverse_wilsons_stacks(g):
    root = randomly_pick_root(g)
    
    tree = nx.Graph()
    tree.add_node(root)
    stacks = initialize_stacks(g, root)
    
    while tree.number_of_nodes() < g.number_of_nodes():
        frontier = get_frontier(g, tree)
        sample = random.choice(frontier)
        neighbor = stacks[sample]
        
        if neighbor in tree.nodes:
            tree.add_edge(sample, neighbor)
        else:
            stacks[sample] = random.choice(list(g.neighbors(sample)))
        
    return tree

for i in range(1000):
    tree = reverse_wilsons_stacks(g)
    test_valid_tree(tree, g)

NameError: name 'g' is not defined

In [4]:
def pop_frontier(g, frontier, stacks):
    for node in frontier:
        stacks[node] = random.choice(list(g.neighbors(node)))
        
def pop_random_node_in_frontier(g, frontier, stacks):
    choice = random.choice(frontier)
    stacks[choice] = random.choice(list(g.neighbors(choice)))

def clear_tree_from_stacks(tree, stacks):
    for node in tree.nodes():
        if node in stacks.keys():
            stacks.pop(node)
            
def add_frontier_to_tree(tree, frontier, stacks):
    to_add = []
    for node in frontier:
        if stacks[node] in tree.nodes:
            to_add.append((node, stacks[node]))
    
    for (u, v) in to_add:
        tree.add_edge(u, v)

def rev_wilsons_stacks(g):
    root = randomly_pick_root(g)
    
    tree = nx.Graph()
    tree.add_node(root)
    stacks = initialize_stacks(g, root)
    
    while tree.number_of_nodes() < g.number_of_nodes():
        frontier = get_frontier(g, tree)
        before = len(tree.nodes)
        add_frontier_to_tree(tree, frontier, stacks)
        clear_tree_from_stacks(tree, stacks)
        after = len(tree.nodes)
        
        if before == after:
            # no new nodes were added
            pop_frontier(g, frontier, stacks)
#             pop_random_node_in_frontier(g, frontier, stacks)
        
    return tree


for i in range(1000):
    tree = rev_wilsons_stacks(g)
    test_valid_tree(tree, g)
    
num_trials = 100000
data = []

for _ in tqdm(range(num_trials)):
    tree = rev_wilsons_stacks(g)

    idx = all_trees.index(tu.tup(tree))
    data.append(idx)
    
tu.plot_sampled_STs(all_trees, data)

NameError: name 'g' is not defined

In [5]:
def add_frontier_to_tree_with_memory(tree, frontier, stacks):
    added_to_tree = []
    for node in frontier:
        if stacks[node] in tree.nodes:
            added_to_tree.append(node)
            tree.add_edge(node, stacks[node])
    
    if len(added_to_tree) > 0:
        for node in added_to_tree:
            frontier.remove(node)
        add_frontier_to_tree_with_memory(tree, frontier, stacks)

def rev_wilsons_with_memory(g):
    root = randomly_pick_root(g)
    
    tree = nx.Graph()
    tree.add_node(root)
    stacks = initialize_stacks(g, root)
    
    while tree.number_of_nodes() < g.number_of_nodes():
        frontier = get_frontier(g, tree)
        before = len(tree.nodes)
        add_frontier_to_tree_with_memory(tree, frontier, stacks)
        after = len(tree.nodes)
        
        if before == after:
            # no new nodes were added
            pop_frontier(g, frontier, stacks)
#             pop_random_node_in_frontier(g, frontier, stacks)
        
    return tree

for i in range(1000):
    tree = rev_wilsons_with_memory(g)
    test_valid_tree(tree, g)
    
num_trials = 100000
data = []

for _ in tqdm(range(num_trials)):
    tree = rev_wilsons_with_memory(g)

    idx = all_trees.index(tu.tup(tree))
    data.append(idx)
    
tu.plot_sampled_STs(all_trees, data)

NameError: name 'g' is not defined

In [6]:
def rev_wilsons_full_boundary(g):
    root = randomly_pick_root(g)
    
    tree = nx.Graph()
    tree.add_node(root)
    stacks = initialize_stacks(g, root)
    
    while tree.number_of_nodes() < g.number_of_nodes():
        frontier = get_frontier(g, tree)
        before = len(tree.nodes)
        add_frontier_to_tree_with_memory(tree, frontier, stacks)
        after = len(tree.nodes)
        
        if before == after:
            # no new nodes were added
            pop_frontier(g, frontier, stacks)
#             pop_random_node_in_frontier(g, frontier, stacks)
        
    return tree

In [7]:
# this is a version of rev wilsons where I try to break the cycles in the frontier if any exist. 
# It does so without generating a new frontier set each time. 
# This is still an idea that could work. 
# I stopped because I was too lazy to figure out how to learn cycles in the frontier online instead of 
# building a graph object offline each time. If I do that then this idea should give a boost in speed.
# I am optimistic of this idea, havent done it yet.

def lerw_rev_wilson(src, g, tree, stacks, frontier):
    curr = src
    branch = nx.Graph()
    reached_root = False

    while reached_root is False:
        nxt = stacks[curr]

        # handle the case of small two edge cycles
        if branch.has_edge(curr, nxt):
            branch.remove_edge(curr, nxt)
            erase_singletons(branch)

            pop(curr, stacks, g)
            pop(nxt, stacks, g)

            curr = nxt
            continue


        branch.add_edge(curr, nxt)
        erase_loop_if_exists(branch, stacks, g)
        curr = nxt

        if nxt in tree.nodes():
            reached_root = True

    tree.update(branch.edges)

    # add the branch's frontier to the tree's frontier
    for node in branch.nodes:
        add_neighbors_to_frontier(node, g, tree, frontier)
    for node in branch.nodes:
        remove_node_from_frontier(node, frontier)
        
def pop_nodes(g, nodes, stacks):
    for node in nodes:
        pop(node, stacks, g)

def add_neighbors_to_frontier(node, g, tree, frontier):
    """ Adds the neighbors of `node" to the frontier.
    """
    for neighbor in g.neighbors(node):
        if neighbor not in tree.nodes():
            frontier.add(neighbor)
            
def remove_node_from_frontier(node, frontier):
    if node in frontier:
        frontier.remove(node)
    
def add_frontier_to_tree(tree, frontier, stacks):
    """ Adds the nodes in the frontier that are pointing
        towards the tree. 
        
        Then, clears the tree from the stacks.
        
        Returns True if nodes were added to the tree, otherwise False.
    """
    # new edges and nodes to be added to the tree respectively
    new_edges = [] 
#     new_nodes = []

#     print("inside func")
#     print("Tree nodes: ", tree.nodes)
#     print("Tree: ", tree.edges)
#     print("Frontier: ", frontier)
    for node in frontier:
        if stacks[node] in tree.nodes:
            new_edges.append((node, stacks[node]))
#             new_nodes.append(node)
    
    # add to tree
    for (u, v) in new_edges:
        tree.add_edge(u, v)
        
#     print("Tree after adding: ", tree.edges)
    # update the frontier
#     for node in new_nodes:
#         add_neighbors_to_frontier(node, g, tree, frontier)
        
#     print("Frontier after adding neighbors: ", frontier)
#     for node in new_nodes:
#         remove_node_from_frontier(node, frontier)
        
#     print("Frontier after node removal: ", frontier)
        
    
    return (len(new_edges) > 0)
            
def get_frontier(g, tree):
    """
    """
    frontier = set()
    for node in tree.nodes:
        for neighbor in g.neighbors(node):
            frontier.add(neighbor)
    frontier = frontier.difference(tree.nodes)
    return frontier


        
def pop_cycles_in_frontier(g, frontier, stacks):
    """ recursively pop cycles from frontier until the frontier has no cycles.
        it returns either a deadlocked frontier, or a frontier
    """
    graph = nx.DiGraph()
    for node in frontier:            
        graph.add_edge(node, stacks[node])
    
    cycles_popped = False
    while loop_exists(graph):
        cycle_nodes = set()
        
        cycle = nx.find_cycle(graph)
        for (u, v) in cycle:
            cycle_nodes.add(u)
            cycle_nodes.add(v)

        pop_nodes(g, cycle_nodes, stacks)
        graph = nx.DiGraph()
        for node in frontier:            
            graph.add_edge(node, stacks[node])
        cycles_popped = True

    return cycles_popped
   
# def get_src_from_outward_nodes(frontier, stacks):
#     outward_nodes = []
#     for node in frontier:
#         if stacks[node] not in frontier:
#             outward_nodes.append(node)
#     return random.choice(outward_nodes)
  
    
def rev_wilsons(g):
    root = randomly_pick_root(g)
    
    tree = nx.Graph()
    tree.add_node(root)
    stacks = initialize_stacks(g, root)
#     frontier = get_frontier(g, tree)
#     add_neighbors_to_frontier(node, g, tree, frontier)
    
    while tree.number_of_nodes() < g.number_of_nodes():
        frontier = get_frontier(g, tree)
        if not add_frontier_to_tree(tree, frontier, stacks):
            # we have hit a deadlocked frontier. 
#             print("hit deadlock")
#             if pop_cycles_in_frontier(g, frontier, stacks):
#                 continue
#             print("k")
            src = pick_leaf(g, tree)
            lerw(src, g, tree, stacks)
#             lerw_rev_wilson(src, g, tree, stacks, frontier)
#             clear_tree_from_stacks(tree, stacks)
#             frontier = get_frontier(g, tree)
    return tree

for i in range(1000):
    tree = rev_wilsons(g)
    tu.test_valid_tree(tree, g)
    
data = []

for _ in tqdm(range(num_trials)):
#     print("NEW")
    tree = rev_wilsons(g)

    idx = all_trees.index(tu.tup(tree))
    data.append(idx)
    
tu.plot_sampled_STs(all_trees, data)

NameError: name 'g' is not defined